# Практика

Выполните клетку ниже для загрузки данных на виртуалку

In [1]:
from google.colab import files
uploaded = files.upload()

Saving sales.csv to sales.csv


In [2]:
# импортируем библиотеки 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Очистка данных: Ticket Sales Data
Набор данных выглядит совсем не очень. В нём есть пропущенные значения, избыточная, и не предназначенная для анализа информация. Ваша задача Привести этот набор данных в приличный вид. Сделайте так, чтобы на этот набор данных было приятно смотреть, и было легко строить на основе него дальнейшую аналитику.

### Задание 1
Прочитайте данные из файла sales.csv в переменную df. Кодировка (encoding) 'latin1'. Обратите также внимание на параметр index_col при чтении.

In [70]:
df = pd.read_csv('sales.csv', encoding='latin1', index_col=0)

### Задание 2
Возьмите 4 случайные записи из набора данных

In [71]:
df.sample(4, axis=0)

,event_id,primary_act_id,secondary_act_id,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,major_cat_name,minor_cat_name,la_event_type_cat,event_disp_name,ticket_text,tickets_purchased_qty,trans_face_val_amt,delivery_type_cd,event_date_time,event_dt,presale_dt,onsale_dt,sales_ord_create_dttm,sales_ord_tran_dt,print_dt,timezn_nm,venue_city,venue_state,venue_postal_cd_sgmt_1,sales_platform_cd,print_flg,la_valid_tkt_event_flg,fin_mkt_nm,web_session_cookie_val,gndr_cd,age_yr,income_amt,edu_val,edu_1st_indv_val,edu_2nd_indv_val,adults_in_hh_num,married_ind,child_present_ind,home_owner_ind,occpn_val,occpn_1st_val,occpn_2nd_val,dist_to_ven
4321,94f673df380d3a39af35,8475a7f6159d0149c11d,f93448eb9d073116037d,dcb1ffc4e578824044e6,Radio 104.5's 9th birthday show,Florence and the Machine,The Lumineers,CONCERTS,ROCK/POP,CONCERTS,Radio 104.5's 9th birthday show,WWW.LIVENATION.COM FLORENCE & THE MAC...,2,20.9,eTicket,2016-06-17 18:00:00,2016-06-17,2016-02-11,2016-02-12,2016-02-11 17:01:51,2016-02-11,NaN,EST,CAMDEN,NEW JERSEY,08103,www.concerts.livenation.com,F,Y,Delaware Valley,dcb1ffc4e578824044e6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
879,b1d121765808836f0344,99b9dbfac8cb6d482013,b85143bf51323b72e53c,49a8c9fb8c82c87b4818,Dancing With The Stars: Live!,Dancing with the Stars,NaN,CONCERTS,ROCK/POP,CONCERTS,Dancing With The Stars: Live!,DANCING WITH THE S...,2,110.0,eTicket,2016-02-06 02:00:00,2016-02-05,2015-10-21,2015-10-26,2015-11-25 02:47:56,2015-11-24,2016-02-03,CST,THACKERVILLE,OKLAHOMA,73459,www.ticketmaster.com,T,Y,Dallas,49a8c9fb8c82c87b4818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2547,8d58e55440725f27b918,f1985939332d998dfb34,b85143bf51323b72e53c,2dc4ce22260d5b032561,"Bryan Adams """"""""""""""""the reckless tour""""""""""""""""",Bryan Adams,NaN,CONCERTS,ROCK/POP,CONCERTS,"Bryan Adams """"""""""""""""the reckless tour""""""""""""""""",LIVE NATION PRESENTS BRYAN ADAMS ...,1,52.0,eTicket,2015-06-29 00:00:00,2015-06-28,2015-05-12,2015-05-15,2015-06-28 14:11:46,2015-06-28,2015-06-28,EST,ST AUGUSTINE,FLORIDA,32080,www.ticketmaster.com,T,Y,Orlando,2dc4ce22260d5b032561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1931,2cd15ebf3ec7b567fe6f,1a3e9aecd0617706a794,b85143bf51323b72e53c,e044a62a7294c72d89ee,Gorge Amphitheatre Camping: Dave Matthews Band...,Gorge Camping,NaN,MISC,SPECIAL ENTRY (UPSELL),UPSELL,Gorge Amphitheatre Camping: Dave Matthews Band...,CAMPGRND OPEN 3PM 8/29/13 OVERNIGHT C A M P I...,1,90.0,Mail,2013-08-29 22:00:00,2013-08-29,NaN,2013-03-22,2013-03-22 17:09:32,2013-03-22,2013-04-04,PST,GEORGE,WASHINGTON,98848,www.concerts.livenation.com,T,N,Seattle,e044a62a7294c72d89ee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Задание 3
Какие данные вы можете выделить из этого набора? 


1.   Ответ оформите в виде 
2.   списка в этой клетке
3.   Название события и ('primary act name'), время события
4.   Категория события и 'minor_cat_name'
5.   Количество купленных билетов и тип доставки
6.   'trans_face_val_amt'
7.   Дата продажи, предпродажи
8.   Город, штат и почтовый индекс события
9.   Платформа продажи
10.   'fin_mkt_nm'

### Задание 4
Проверьте набор данных на явно дублирующие столбцы и дубликаты в записях, удалите их

In [72]:
df = df.drop_duplicates()
df = df.T.drop_duplicates().T

### Задание 5
Есть ли столбцы, которые одинаковы по смыслу, но фактически в них разные записи? Если такие столбцы есть, оставьте какой-то один из них, либо сформируйте один столбец на основе нескольких.

In [73]:
df_replace_date = df[df['sales_ord_create_dttm'].isna()].apply(lambda x: x['sales_ord_tran_dt'] + ' 00:00:00', axis=1)
df.loc[df_replace_date.index, 'sales_ord_create_dttm'] = df_replace_date 
#df.apply(lambda x: x['venue_city'] + ',' + x['venue_state'], axis=1)

In [74]:
df.drop(['event_disp_name', 'la_event_type_cat', 'event_dt', 'sales_ord_tran_dt'], 1, inplace=True)

### Задание 6
Все ли типы данных оптимальны? Приведите колонки к корректным типам данных. Например, в наборе имеются несколько столбцов представляющих дату

In [75]:
df = df.astype({'event_date_time': 'datetime64',
           'presale_dt': 'datetime64[D]',
           'onsale_dt': 'datetime64[D]',
           'sales_ord_create_dttm': 'datetime64',
           'print_dt': 'datetime64[D]',
           'major_cat_name': 'category',
           'tickets_purchased_qty': 'int8',
           'trans_face_val_amt': 'float16',
           'delivery_type_cd': 'category',
           'timezn_nm': 'category',
           'print_flg': 'category',
           'la_valid_tkt_event_flg': 'category'})

### Задание 7
Посмотрите уникальные значения столбца sales_platform_cd

In [76]:
df['sales_platform_cd'].unique()

array(['www.concerts.livenation.com', nan, 'www.livenation.com',
       'mobile.livenation.us', 'www.ticketmaster.com',
       'iphone.ticketmaster.us', 'android.ticketmaster.us',
       'iphone2.livenation.us', 'mobile.ticketmaster.us',
       'www.ticketmaster.ca', 'mobile.ticketmaster.ca',
       'androidphone2.livenation.us', 'iphone.ticketmaster.ca',
       'www.ticketmaster.com.mx'], dtype=object)

### Задание 8
Разделите столбец sales_platform_cd на два столбца. сайт где был куплен билет, и платформу которая была использована при покупке. Здесь сайт это, например, ticketmaster.com, а платформа это www, www2, iphone, mobile итд

In [77]:
df_s_platf = df[~df['sales_platform_cd'].isna()]['sales_platform_cd'].str.split('.', n=1, expand=True)
df_s_platf.columns = ['sale_platform', 'sale_site']
df = pd.concat([df, df_s_platf], axis=1).drop('sales_platform_cd', axis=1)

### Задание 9
Судя по присутствию слов PARKING и прочее не все билеты здесь являются билетами на мероприятия. Найдите в данных столбец, наиболее точно отражающий тип билета. Используя этот столбец оставьте только билеты на мероприятия, т.е уберите все INVALID/PARKING/... билеты из данных

In [78]:
df.drop(df.index[df.loc[:,'minor_cat_name'] == 'PARKING'], axis=0, inplace=True)
df.drop(df.index[df.loc[:,'minor_cat_name'] == 'DINNER PACKAGES (UPSELL)'], axis=0, inplace=True)
df.drop(df.index[df.loc[:,'minor_cat_name'] == 'CONCESSION VOUCHERS (UPSELL)'], axis=0, inplace=True)
df.drop(df.index[df.loc[:,'minor_cat_name'] == 'CAMPING'], axis=0, inplace=True)
df.drop(df.index[df['ticket_text'].str.find('NOT VALID') != -1], axis=0, inplace=True)
df.drop(df.index[df['ticket_text'].str.find('THIS IS NOT A TICKET') != -1], axis=0, inplace=True)
df.drop(df.index[df['ticket_text'].str.find('NOT A CONCERT TICKET') != -1], axis=0, inplace=True)
df.drop(df.index[df['ticket_text'].str.find('EVENT TICKET REQUIRED') != -1], axis=0, inplace=True)
df.drop(df.index[df['ticket_text'].str.find('NOT AN EVENT TICKET') != -1], axis=0, inplace=True)
df.drop(df.index[df['ticket_text'].str.find('THIS IS NOT A CONCERT TKT') != -1], axis=0, inplace=True)
df.drop(df.index[df['ticket_text'].str.find('WITH EVENT TICKET') != -1], axis=0, inplace=True)


### Задание 10
Подберите для столбцов более "человеческие" названия и переименуйте их (как будто собираетесь скинуть эту таблицу кому-то, далекому от технической стороны вопроса)

In [79]:
df.rename({'major_cat_name': 'major_category_name',
           'minor_cat_name': 'minor_category_name',
           'tickets_purchased_qty': 'ticket_purchased_quantity',
           'trans_face_val_amt': 'cost',
           'delivery_type_cd': 'delivery_type',
           'presale_dt': 'presale_date',
           'onsale_dt': 'onsale_date',
           'sales_ord_create_dttm': 'sales_ord_create_date_time',
           'print_dt': 'print_date',
           'timezn_nm': 'timezone_name',
           'venue_postal_cd_sgmt_1': 'venue_postcode',
           'gndr_cd': 'gender',
           'age_yr': 'age_year',
           'dist_to_ven': 'distance_to_venue'}, axis=1, inplace=True)

### Задание 11
Ответьте на следующие вопросы
1.  Какой наиболее популярный жанр концерта?
2.  Сколько в среднем продается билетов за одну покупку?
3.  Какой наиболее популярный сайт для покупки билетов?

In [83]:
#1.df['minor_category_name'].value_counts().index[0]
#2.df['ticket_purchased_quantity'].mean()
#3.df['sale_site'].value_counts().index[0]